Rede Neural Recorrente

Prevendo próximos dias de variações de ações do Google

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Criando o dataset e preenchendo os valores

In [3]:
dataset_train = pd.read_csv('Google_Stock_Price_Train.csv')
training_set =dataset_train.iloc[:,1:2].values

Realizando Featuring Scalling

In [4]:
from sklearn.preprocessing import MinMaxScaler
                                               
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(training_set)

Criando a estrutura de dados com os intervalos de tempos, para prever o dia seguinte

In [5]:
X_train =[]
y_train= []



Populando os dados com 60 dias

In [6]:
for i in range (60,1258):
  X_train.append(training_set_scaled[i-60:i,0])
  y_train.append(training_set_scaled[i:0])

X_train, y_train = np.array(X_train),np.array(y_train)

COlocando os dados nos formador do Keras (três dimensões)

In [7]:
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))

In [10]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout


regressor = Sequential()



Criando as camadas

In [12]:
#camada de entrada
regressor.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],1)))
regressor.add(Dropout(0.2))
#segunda camada
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))
#terceira camada
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))
#quarta camada
regressor.add(LSTM(units=50))
regressor.add(Dropout(0.2))

#saida camada
regressor.add(Dense(units=1))


Compilando a rede neural

In [16]:
regressor.compile(optimizer='adam', loss='mean_squared_error')



In [ ]:
regressor.fit(X_train, y_train, epochs = 100, batch_size=32)

In [20]:
dataset_test = pd.read_csv('Google_Stock_Price_Test.csv')
real_stock_price=dataset_test.iloc[:,1:2].values

concatenando os dados de treino e de teste

In [22]:
dataset_total = pd.concat((dataset_train['Open'], dataset_test['Open']),axis=0)

intervalo de 60 dias anteriores

In [24]:
inputs= dataset_total[len(dataset_total)-len(dataset_test)-60:].values

In [25]:
inputs = inputs.reshape(-1,1)

Normalizando

In [27]:
inputs = sc.transform(inputs)
X_test =[]


In [29]:
for i in range(60,80):
  X_test.append(inputs[i-60:i,0])
X_test =np.array(X_test)

X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1],1))

AttributeError: ignored

Fazendo a predição do mês de Janeiro

In [ ]:
predicted_stock_price = regressor.predict(X_test)

Gerando o gráfico pra comparar a previsão do que de fato aconteceu

In [ ]:
plt.plot(real_stock_price, color ='red', label='Dados Reais de Ações do Google')
plt.plot(predicted_stock_price, color ='blue', label='Dados Previstos de Ações do Google')
plt.title('Previsao de Preços de Ações')
plt.xlabel('Tempo')
plt.ylabel('Preços de Ações do Google')
plt.legend()
plt.show()
